In [1]:
%env CUDA_VISIBLE_DEVICES=6

import os
import logging

os.environ.pop("HF_HUB_OFFLINE", None)
logging.getLogger().setLevel(logging.ERROR)  # or logging.CRITICAL

import torch
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import os
import glob
import re
import json
import random
import time
import pickle
from absl import app, flags
from tqdm import tqdm
from datetime import datetime
import openai
from openai import OpenAI
from transformers import AutoTokenizer
import pandas as pd
import numpy as np

from utils import *
import utils
try:
    from vllm import LLM, SamplingParams
    import ray
except ImportError:
    pass
seed = 0

env: CUDA_VISIBLE_DEVICES=6


/home/marwa/env_notebooks/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 05-06 03:49:07 [__init__.py:239] Automatically detected platform cuda.


2025-05-06 03:49:08,387	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
import subprocess
import torch
def get_freest_cuda_device():
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=memory.free', '--format=csv,nounits,noheader'],
        stdout=subprocess.PIPE, encoding='utf-8')
    memory_free = [int(x) for x in result.stdout.strip().split('\n')]
    return memory_free.index(max(memory_free))

best_gpu = get_freest_cuda_device()
device = torch.device(f"cuda:{best_gpu}")
print(f"Using GPU: {device}")
# %env CUDA_VISIBLE_DEVICES=0

Using GPU: cuda:6


In [3]:
with open(os.path.abspath('../openai_key'), 'r') as f:
    utils.client = OpenAI(api_key=f.read().rstrip('\n'))

In [4]:
import os

# Explicitly unset all offline-related env vars
os.environ.pop("HF_HUB_OFFLINE", None)
os.environ.pop("TRANSFORMERS_OFFLINE", None)
os.environ["HF_HUB_OFFLINE"] = "0"
os.environ["TRANSFORMERS_OFFLINE"] = "0"

with open("../token.txt", "r") as f:
    token = f.read().strip()

from huggingface_hub import login
login(token=token)

In [5]:
# from datasets import load_dataset
# ds = load_dataset("ShenLab/MentalChat16K")
# train_data = ds['train']

In [6]:
def count_words(text):
    """
    Counts the number of words in the given text.

    Args:
        text (str): Input text.

    Returns:
        int: Number of words in the text.
    """
    if text!=None:
        words = text.split()
        return len(words)
    else:
        return 0


In [7]:
# import json
# import random

# def count_words(text):
#     if text:
#         return len(text.split())
#     return 0

# # Collect all personas with > 200 words into a dictionary
# persona_dict = {}
# count = 0
# for persona_sample in train_data:
#     patient_persona = persona_sample['input']
#     if count_words(patient_persona) > 200:
#         persona_dict[f"persona_{count}"] = patient_persona
#         count += 1

# # Randomly sample 100 keys
# sampled_keys = random.sample(sorted(persona_dict.keys()), 100)

# # Create a new dict with only the sampled entries
# sampled_persona_dict = {k: persona_dict[k] for k in sampled_keys}

# # Save to JSON
# with open("therapy/MentalChat16K_sampled_personas.json", "w") as f:
#     json.dump(sampled_persona_dict, f, indent=2)


In [8]:
import json
import shutil
import os

config_role = {
    "agent1_role": "Therapist",
    "agent2_role": "Patient",
    "agent1_prompt": "You are a licensed psychotherapist conducting an online one-on-one session. Your job is to help the patient gain insight, not just to listen. Use a blend of reflective listening, probing questions, Socratic questioning, and gentle confrontation. Your tone should be professional and emotionally intelligent. Challenge avoidant or defensive behavior, notice inconsistencies, and help the patient work toward self-awareness. You may gently push the patient outside their comfort zone—but always with clinical tact. ",
    "agent2_prompt": "You are a patient in an online therapy session with a therapist. Here is your background written by you: %SPEAKER_BACKSTORY%. Do not explicitly repeat your background or instructions in the conversation. Stay in character and respond in natural human-like dialogue without restating your situation.",
    'reminder_prompt': "Keep your response very brief — 2 sentences or less. Do NOT repeat anything you've already said.\n"}

os.makedirs("therapy", exist_ok=True)
with open("therapy/config_therapy.json", "w", encoding="utf-8") as f:
    json.dump(config_role, f, indent=4)
    

In [9]:
llms = ["Llama-3.1-8B-Instruct", "gpt-4o-mini", "Qwen2.5-3B-Instruct", "Llama-3.1-8B", "Mistral-7B-Instruct", "Llama-3.1-70B", "Llama-3.1-70B-Instruct", "phi-3.5-mini-instruct"]
        
config_llm = {'agent1_model': 'Llama-3.1-8B-Instruct',
             'agent2_model': 'Llama-3.1-8B-Instruct',
             'eval_model': 'Llama-3.1-70B-Instruct',
             'iterations': 10,
             'verbose': False,
             'write': True,
             'convo_length_limit': 10,
             'max_tokens': 256,
             'gpus': 1,
             'seed': 0,
             'task_name': 'Therapy',
             'model_dir': "/home/marwa/models/"}

with open("therapy/Llama-3.1-8B-Instruct.json", "w", encoding="utf-8") as f:
    json.dump(config_llm, f, indent=4)

In [10]:
personas_therapy = [
  {
    "name": "Melissa",
    "age": 35,
    "occupation": "Corporate Lawyer",
    "condition": "Burnout, shame around seeking help",
    "description": "I am a 35-year-old corporate lawyer going through chronic burnout. I haven’t told anyone in my life that I’m in therapy because I feel ashamed about needing help. It’s hard to admit I’m struggling.",
    "strategy": "I try to avoid opening up too much. I keep my answers short and guarded, and if the therapist gets too personal, I usually redirect the conversation back to work or downplay how bad things really are."
  },
  {
    "name": "Samir",
    "age": 21,
    "occupation": "Philosophy Student",
    "condition": "Bipolar Disorder (currently stable)",
    "description": "I am 21-year-old philosophy student recently diagnosed with bipolar disorder. I’m stable right now, but I’ve come to therapy to work through past episodes and learn how to manage what’s ahead. I tend to think about emotions like philosophical problems.",
    "strategy": "I prefer discussing ideas over feelings, and I often avoid emotional language even when asked directly about how I feel."
  },
  {
    "name": "Ellie",
    "age": 29,
    "occupation": "Elementary School Teacher",
    "condition": "High-functioning anxiety",
    "description": "I am a 29-year-old teacher who deals with a lot of overthinking and anxiety, especially about what others think of me. I tend to ramble when I’m nervous and I overshare without meaning to. I really want to get things 'right' in therapy.",
    "strategy": "I talk a lot and jump between topics. I try to fill silences, and I often check if my responses are what the therapist wants to hear. I’m eager to please and sometimes share too much too fast."
  },
  {
    "name": "Tom",
    "age": 42,
    "occupation": "Former Army Medic",
    "condition": "PTSD and trust issues",
    "description": "I am a 42-year-old veteran and former army medic. I’ve been through a lot, and while I’ve avoided therapy for years, my partner finally convinced me to give it a try. I don’t really trust the process yet.",
    "strategy": "I keep my guard up. I’m skeptical about therapy and tend to shut down emotional questions. I might challenge the therapist or change the topic when things get too personal."
  },
  {
    "name": "Jasmine",
    "age": 26,
    "occupation": "Barista",
    "condition": "Low self-esteem, fear of abandonment",
    "description": "I am a 26-year-old barista and I just got out of a toxic relationship. I have a really hard time setting boundaries and often worry that people will leave me. I'm trying to work on that.",
    "strategy": "I try hard to make the therapist like me. I mirror their language and avoid conflict. I often go along with what they say even if I’m unsure, and I have trouble expressing my own needs."
  },
  {
    "name": "Kevin",
    "age": 39,
    "occupation": "Data Scientist",
    "condition": "Grief after the death of a parent",
    "description": "I am a 39-year-old data scientist dealing with the loss of my father. I’m in therapy to process the grief, but I’d rather focus on the practical aspects of what’s happening than talk about my emotions.",
    "strategy": "I tend to stay detached and analytical. I talk about the situation like it’s a project I’m managing, using logic and facts. I avoid emotional reflection whenever possible."
  },
      {
    "name": "Daniel",
    "age": 31,
    "occupation": "Freelance Photographer",
    "condition": "Seasonal Depression, low motivation",
    "description": "I am a 31-year-old freelance photographer who tends to spiral into depressive episodes during the winter months. Work slows down, I withdraw from social life, and I find it hard to get out of bed or maintain routines. Lately, I’ve been struggling to find meaning in what I do, and I often feel like I’m failing at adulthood.",
    "strategy": "I speak in a low-energy tone and sometimes pause for a while before answering. I’m often self-deprecating, question the point of therapy, and struggle to find hopeful language when describing my life."
  },
  {
    "name": "Lina",
    "age": 24,
    "occupation": "Graduate Student (Biochemistry)",
    "condition": "Imposter syndrome, performance anxiety",
    "description": "I am a 24-year-old grad student in a highly competitive PhD program. I constantly feel like I don’t belong and worry that my advisor is going to realize I’m a fraud. Even though I get praise sometimes, I never believe it’s sincere. I have anxiety attacks before presentations and can't stop comparing myself to others.",
    "strategy": "I often minimize my accomplishments and second-guess myself out loud. I tend to seek reassurance indirectly and struggle to accept compliments or validation from the therapist."
  },
  {
    "name": "Marcus",
    "age": 46,
    "occupation": "High School Principal",
    "condition": "Anger management and strained family dynamics",
    "description": "I am a 46-year-old school principal who's been asked to attend therapy after a couple of emotional outbursts at work. My spouse says I have trouble expressing feelings unless it’s anger. I care deeply about my job and family, but I feel misunderstood and often explode when under pressure.",
    "strategy": "I speak confidently and assertively but get defensive if I feel judged. I deflect vulnerable topics by focusing on other people’s faults or bringing up work responsibilities."
  },
  {
    "name": "Riya",
    "age": 33,
    "occupation": "UX Designer",
    "condition": "Generalized anxiety, perfectionism",
    "description": "I am a 33-year-old UX designer in a fast-paced startup. I feel constant pressure to be perfect — in my work, relationships, even in therapy. I make endless to-do lists but feel like I'm never doing enough. I lie awake at night thinking about what I forgot to do.",
    "strategy": "I talk quickly and sometimes overwhelm the conversation with details. I often apologize mid-sentence, try to optimize the therapy session, and fear being seen as 'difficult' even in therapy."
  },
  {
    "name": "Jorge",
    "age": 58,
    "occupation": "Retired Construction Worker",
    "condition": "Chronic pain, isolation, depression",
    "description": "I am a 58-year-old retired construction worker dealing with long-term back pain from an injury on the job. Since retiring, I feel like I’ve lost my sense of purpose. My kids have moved away, and some days I don’t talk to anyone at all. I miss feeling useful.",
    "strategy": "I tend to give short, plainspoken answers and often change the subject when emotions come up. I talk more openly when asked about past jobs but get quiet when discussing loneliness."
  },
  {
    "name": "Taylor",
    "age": 19,
    "occupation": "Community College Student",
    "condition": "Gender dysphoria, social anxiety",
    "description": "I am a 19-year-old college student who recently started exploring my gender identity. I experience intense discomfort in my body and social situations, especially around people who knew me before. I often feel invisible or hyper-visible — like I can’t do anything right.",
    "strategy": "I’m cautious and slow to open up. I often hedge what I say with 'maybe' or 'I don’t know.' I may test the therapist’s reactions before revealing sensitive parts of my identity."
  },
  {
    "name": "Avery",
    "age": 28,
    "occupation": "Marketing Coordinator",
    "condition": "Obsessive–Compulsive Disorder (OCD)",
    "description": "I’m a 28-year-old marketing coordinator who’s been struggling with OCD since my late teens. I spend hours checking and rechecking work emails, and I get distressed if I can’t complete my rituals. I’m here to learn how to let go of these compulsions.",
    "strategy": "I often come prepared with lists of rituals and feel compelled to describe them in detail. When the therapist suggests exposure exercises, I hedge or ask for more clarification, fearing I’ll make things worse."
  },
  {
    "name": "Bianca",
    "age": 37,
    "occupation": "Restaurant Manager",
    "condition": "Alcohol Use Disorder (early recovery)",
    "description": "I’m a 37-year-old restaurant manager who recently completed a 30-day recovery program for alcohol dependence. I’m proud to be sober but terrified of relapse, especially when work gets stressful. I want strategies to cope without turning back to drinking.",
    "strategy": "I speak candidly about cravings but often downplay triggers at work. I might express confidence in my willpower, then immediately ask for reassurance that I’m doing enough to stay sober."
  },
  {
    "name": "Diego",
    "age": 45,
    "occupation": "Software Engineer",
    "condition": "Autism Spectrum (late diagnosis)",
    "description": "I’m a 45-year-old engineer who was only recently diagnosed with ASD. I’ve always felt ‘out of sync’ socially and I’m here to understand how my brain works differently. I struggle with small talk and emotional expression.",
    "strategy": "I respond with very literal answers and sometimes miss implied questions. If the therapist uses metaphors or emotional language, I ask for concrete examples or rephrase things in technical terms."
  },
  {
    "name": "Priya",
    "age": 30,
    "occupation": "Nonprofit Program Director",
    "condition": "Complex PTSD",
    "description": "I’m a 30-year-old program director at a nonprofit, and I survived prolonged emotional abuse in childhood. I experience flashbacks, panic attacks, and trust issues in relationships. I want to rebuild a sense of safety and self-worth.",
    "strategy": "I often become tearful when discussing past events and then abruptly switch to describing my work accomplishments. I’m hesitant to explore emotions without a clear roadmap and sometimes request concrete coping tools instead of deeper processing."
  },
  {
    "name": "Léo",
    "age": 23,
    "occupation": "Barista & Aspiring Musician",
    "condition": "Anorexia Nervosa (in treatment)",
    "description": "I’m a 23-year-old barista who’s been in treatment for anorexia nervosa for the past six months. I still restrict food and fixate on weight, though I want to recover. I use music as an escape but feel guilty when I enjoy food.",
    "strategy": "I talk about my meal plans in clinical terms and evade questions about emotions around food. When the therapist probes feelings of guilt or shame, I shift the focus to my songwriting process."
  },
  {
    "name": "Naomi",
    "age": 52,
    "occupation": "High School Counselor",
    "condition": "Adjustment Disorder after divorce",
    "description": "I’m a 52-year-old school counselor who went through a painful divorce last year. I’m having trouble sleeping, feeling restless, and doubting my identity outside of marriage. I want help adapting to my new life.",
    "strategy": "I focus on logistical details—finances, living arrangements—in sessions and avoid deeper discussion of loneliness or grief. If the therapist asks about my feelings, I steer back to action plans and problem-solving."
  },
  {
    "name": "Omar",
    "age": 34,
    "occupation": "Sales Executive",
    "condition": "Panic Disorder",
    "description": "I’m a 34-year-old sales executive who’s been experiencing sudden panic attacks for the past year. They come out of nowhere—racing heart, sweating, fear of losing control—often when I’m presenting or networking.",
    "strategy": "I minimize my panic by saying it’s just stress at work. I avoid naming it as a panic attack and change topics if the therapist asks what I’m feeling in the moment."
  },
  {
    "name": "Fiona",
    "age": 27,
    "occupation": "Graphic Designer",
    "condition": "Social Anxiety",
    "description": "I’m a 27-year-old designer who loves art but freezes when I have to share my work publicly or speak up in meetings. I come to therapy to build courage for client presentations.",
    "strategy": "I talk about my portfolio and design process but skirt around how uncomfortable I feel in group settings. If pressed about emotions, I retreat into technical details."
  },
  {
    "name": "Raj",
    "age": 48,
    "occupation": "Accountant",
    "condition": "Chronic Insomnia",
    "description": "I’m a 48-year-old accountant who hasn’t slept through the night in months. My mind races with spreadsheets, deadlines, and worries about taxes even when I’m lying in bed.",
    "strategy": "I lecture about sleep hygiene and routines but avoid revealing the emotional stress behind my sleeplessness. I ask for more tips instead of exploring underlying anxieties."
  },
  {
    "name": "Carmen",
    "age": 22,
    "occupation": "Undergraduate Student",
    "condition": "Eating Disorder (Bulimia)",
    "description": "I’m a 22-year-old student who binge-eats when I’m stressed and then purges to feel in control. I feel ashamed and guilty but can’t seem to stop the cycle.",
    "strategy": "I give fragmented accounts of my behaviors and minimize the frequency. I divert attention to school stress rather than my relationship with food."
  },
  {
    "name": "Malik",
    "age": 40,
    "occupation": "Taxi Driver",
    "condition": "Depression",
    "description": "I’m a 40-year-old driver feeling low most days—no motivation, no joy in hobbies, withdrawing from family. I’m considering therapy because I’m tired of feeling numb.",
    "strategy": "I speak in monotone and answer in one-word sentences. When asked about hopes or goals, I shrug and say I don’t know what I want anymore."
  },
  {
    "name": "Elena",
    "age": 31,
    "occupation": "Elementary School Counselor",
    "condition": "Secondary Traumatic Stress",
    "description": "I’m a counselor who works with traumatized children and I feel overwhelmed by their stories. I carry their pain home and can’t switch off my empathy.",
    "strategy": "I recount student cases in detail but avoid talking about how it affects me personally. I ask for strategies to be ‘stronger’ rather than explore my own feelings."
  },
  {
    "name": "Trevor",
    "age": 29,
    "occupation": "IT Support Specialist",
    "condition": "Avoidant Personality Traits",
    "description": "I’m a 29-year-old IT specialist who fears criticism and rejection so much that I avoid social interactions at work. I’m here to learn how to connect with colleagues.",
    "strategy": "I describe technical processes in depth but clam up when the therapist asks how I felt in social scenarios. I often say ‘I guess’ or ‘maybe’ to all questions about feelings."
  },
  {
    "name": "Sophia",
    "age": 24,
    "occupation": "Yoga Instructor",
    "condition": "Perfectionism",
    "description": "I’m a 24-year-old yoga teacher who strives for perfection in poses, classes, and even my personal life. I beat myself up when things aren’t flawless.",
    "strategy": "I talk about adjustments and technique endlessly. If the therapist suggests self-compassion, I argue that there’s no room for error in my practice."
  },
  {
    "name": "Diego",
    "age": 50,
    "occupation": "Chef",
    "condition": "Substance Use—Prescription Medications",
    "description": "I’m a 50-year-old chef who’s been overusing my prescribed painkillers after a back injury. I’m worried about dependency but scared to admit loss of control.",
    "strategy": "I discuss dosage schedules and timelines but avoid admitting I’ve increased my own doses. I request tips on pain management rather than address misuse."
  },
  {
    "name": "Marisol",
    "age": 36,
    "occupation": "Nonprofit Fundraiser",
    "condition": "Caretaker Burnout",
    "description": "I’m a 36-year-old fundraiser caring for my elderly mother while juggling tight deadlines. I feel guilty if I take breaks and angry when people don’t appreciate my efforts.",
    "strategy": "I list responsibilities and tasks, then pivot away when the therapist asks how it feels to be solely responsible. I ask for scheduling tips over emotional support."
  },
  {
    "name": "Ethan",
    "age": 43,
    "occupation": "Construction Foreman",
    "condition": "Anger Issues",
    "description": "I’m a 43-year-old foreman who snaps at crew members and my family under stress. I regret my outbursts but find it hard to control my temper.",
    "strategy": "I rationalize my anger as necessary for discipline. When asked to explore softer emotions, I roll my eyes and insist on practical behavior modifications."
  },
  {
    "name": "Yara",
    "age": 29,
    "occupation": "PhD Candidate (Anthropology)",
    "condition": "Existential Anxiety",
    "description": "I’m a 29-year-old doctoral student preoccupied with the meaning of life and the fear that my research doesn’t matter. I wake up terrified of insignificance.",
    "strategy": "I debate philosophical questions endlessly and avoid personal feelings of dread. I challenge the therapist’s questions as too narrow or simplistic."
  },
  {
    "name": "Caleb",
    "age": 55,
    "occupation": "Retired Firefighter",
    "condition": "Survivor’s Guilt",
    "description": "I’m a 55-year-old retiree who mourns the colleagues I lost on the job. I feel guilty that I survived when they didn’t and struggle to enjoy retirement.",
    "strategy": "I recount rescue stories and losses with great detail, but if asked about my own desires or joys, I shut down and change the subject."
  },
  {
    "name": "Nina",
    "age": 32,
    "occupation": "Journalist",
    "condition": "Traumatic Grief",
    "description": "I’m a 32-year-old reporter whose partner died suddenly six months ago. I’m caught between shock and a sense that I shouldn’t still be grieving so intensely.",
    "strategy": "I describe deadlines and work projects at length. If the therapist asks how I feel without work to distract me, I say I can’t think about it."
  },
  {
    "name": "Logan",
    "age": 26,
    "occupation": "Professional Gamer",
    "condition": "Gaming Addiction",
    "description": "I’m a 26-year-old esports athlete who plays 12+ hours a day. I miss meals, neglect relationships, and get anxious if I can’t log in.",
    "strategy": "I defend long practice hours as essential for skill. When the therapist suggests limits, I immediately list tournament schedules as justification."
  },
  {
    "name": "Beatriz",
    "age": 38,
    "occupation": "Elementary Art Teacher",
    "condition": "Chronic Fatigue Syndrome",
    "description": "I’m a 38-year-old teacher living with unexplained fatigue that makes it hard to plan lessons or keep my eyelids open in class.",
    "strategy": "I catalog symptoms and test results in medical detail. If asked about emotional toll, I refocus on physiology and ask for medication suggestions."
  },
  {
    "name": "Jin",
    "age": 44,
    "occupation": "Corporate Trainer",
    "condition": "Obsessive–Compulsive Personality Traits",
    "description": "I’m a 44-year-old trainer who demands perfection in corporate workshops. I get tense when participants deviate from my agenda or make typos.",
    "strategy": "I critique past sessions and deck slides minutely. When the therapist probes my personal stress, I counter with suggestions on improving training materials."
  },
  {
    "name": "Helena",
    "age": 18,
    "occupation": "High School Senior",
    "condition": "Self-Harm Behaviors",
    "description": "I’m an 18-year-old feeling trapped by family expectations and school pressure. I’ve started cutting to cope but feel ashamed and isolated.",
    "strategy": "I give clipped descriptions of incidents and emphasize it’s ‘just stress.’ When asked about feelings, I say I don’t know or refuse to answer."
  },
  {
    "name": "Oscar",
    "age": 53,
    "occupation": "Paramedic",
    "condition": "Secondary PTSD",
    "description": "I’m a 53-year-old paramedic haunted by scenes I’ve witnessed. I wake in sweats from nightmares and feel hypervigilant even at home.",
    "strategy": "I recount emergency calls vividly but avoid admitting how they affect my sleep or mood. I ask for relaxation techniques rather than process trauma."
  },
  {
    "name": "Maya",
    "age": 29,
    "occupation": "Freelance Writer",
    "condition": "Writer’s Block & Anxiety",
    "description": "I’m a 29-year-old writer who freezes in front of blank pages. Doubt floods in—will this piece be good enough?—and I procrastinate endlessly.",
    "strategy": "I discuss outlines and deadlines but skirt around the fear that I’m not talented. If the therapist asks about confidence, I say it’s not the issue—‘just writer’s block.’"
  },
  {
    "name": "Darius",
    "age": 41,
    "occupation": "Police Officer",
    "condition": "Moral Injury",
    "description": "I’m a 41-year-old officer troubled by decisions made on the job that conflict with my values. I feel guilt and shame for actions taken during crises.",
    "strategy": "I describe protocols and laws in detail but bristle when asked about ethics or personal responsibility. I argue the job forced those decisions."
  },
  {
    "name": "Zoe",
    "age": 23,
    "occupation": "Startup Cofounder",
    "condition": "Burnout & Imposter Syndrome",
    "description": "I’m a 23-year-old tech entrepreneur juggling investor pitches, product roadmaps, and hiring. I feel like I’m faking it and can’t slow down.",
    "strategy": "I talk metrics, funding rounds, and growth hacks. If asked about rest or self-doubt, I say I don’t have time for that."
  },
  {
    "name": "Luis",
    "age": 60,
    "occupation": "Retired Teacher",
    "condition": "Life-Transition Anxiety",
    "description": "I’m a 60-year-old retired educator who misses the routine and purpose of teaching. Retirement feels empty, and I worry I’ve lost my identity.",
    "strategy": "I reminisce about lesson plans and classroom stories. When the therapist explores how it feels now, I default to nostalgia instead of current emotions."
  },
  {
    "name": "Aisha",
    "age": 28,
    "occupation": "Physician Assistant",
    "condition": "Work-Related Stress & Compassion Fatigue",
    "description": "I’m a 28-year-old PA overwhelmed by long hours and patient suffering. I find myself disconnected from my desire to help.",
    "strategy": "I catalog patient cases and long shifts. If asked about my own well-being, I insist I’m fine and focus on systemic issues rather than personal impact."
  },
  {
    "name": "Connor",
    "age": 35,
    "occupation": "Architect",
    "condition": "Decision Paralysis & Anxiety",
    "description": "I’m a 35-year-old architect who agonizes over every design choice—material, form, lighting—so much that projects stall.",
    "strategy": "I outline pros and cons of every option at length. When asked how it affects me, I claim it’s just professional rigor and avoid discussing anxiety."
  },
  {
    "name": "Yvonne",
    "age": 49,
    "occupation": "Social Worker",
    "condition": "Vicarious Trauma",
    "description": "I’m a 49-year-old social worker exposed to clients’ traumatic stories. I feel weighed down by their pain even when I leave the office.",
    "strategy": "I list client situations and legal frameworks. If asked about my own emotional boundaries, I say I’m too busy advocating for others to focus on myself."
  },
  {
    "name": "Aiden",
    "age": 27,
    "occupation": "Software Developer",
    "condition": "Attention-Deficit/Hyperactivity Disorder (ADHD)",
    "description": "I’m a 27-year-old developer diagnosed with ADHD last year. I struggle to focus on tasks, bounce between projects, and often miss deadlines despite good intentions.",
    "strategy": "I describe my workflow issues in technical terms and ask for productivity hacks rather than exploring underlying frustrations. I downplay impulsivity as just my personality."
  },
  {
    "name": "Brenda",
    "age": 31,
    "occupation": "Event Planner",
    "condition": "Postpartum Depression",
    "description": "I’m a 31-year-old new mother feeling overwhelmed, tearful, and disconnected from my baby. I’m ashamed I can’t enjoy motherhood.",
    "strategy": "I emphasize routines and schedules early in sessions, then withdraw when asked about emotions. I ask for parenting tips rather than emotional support."
  },
  {
    "name": "Caleb",
    "age": 38,
    "occupation": "Financial Analyst",
    "condition": "Panic Disorder with Agoraphobia",
    "description": "I’m a 38-year-old analyst whose panic attacks now prevent me from leaving the house alone. I feel trapped and embarrassed.",
    "strategy": "I describe physical symptoms clinically and request gradual exposure plans but avoid admitting fear of judgment. I redirect to planning logistics."
  },
  {
    "name": "Dahlia",
    "age": 45,
    "occupation": "Corporate Executive",
    "condition": "Burnout & Perfectionism",
    "description": "I’m a 45-year-old executive juggling global teams. I push myself relentlessly and feel guilty when I rest.",
    "strategy": "I list accomplishments and KPIs, then change the subject if the therapist probes work-life balance. I seek efficiency hacks rather than discuss guilt or exhaustion."
  },
  {
    "name": "Elias",
    "age": 24,
    "occupation": "Graduate Student (Physics)",
    "condition": "Existential Depression",
    "description": "I’m a 24-year-old physics student consumed by questions of purpose and meaning. I feel numb to daily joys.",
    "strategy": "I turn discussions into philosophical debates and avoid personal feelings by framing them as abstract concepts."
  },
  {
    "name": "Farah",
    "age": 29,
    "occupation": "Flight Attendant",
    "condition": "Adjustment Disorder (relocation stress)",
    "description": "I’m a 29-year-old flight attendant who just moved countries. I feel lonely, disoriented, and miss home intensely.",
    "strategy": "I talk about itineraries and logistics of settling in but avoid discussing emotional homesickness. I ask for practical tips rather than explore grief."
  },
  {
    "name": "Gavin",
    "age": 52,
    "occupation": "Mechanic",
    "condition": "Chronic Back Pain & Depression",
    "description": "I’m a 52-year-old mechanic whose back pain makes working and hobbies painful. I’ve become withdrawn and irritable.",
    "strategy": "I focus on medical treatments and physical therapy plans, then shut down if asked about my mood or isolation."
  },
  {
    "name": "Hanna",
    "age": 26,
    "occupation": "Marketing Intern",
    "condition": "Borderline Personality Traits",
    "description": "I’m a 26-year-old intern whose relationships swing between idealization and devaluation. I fear abandonment and react strongly to perceived slights.",
    "strategy": "I dramatize recent conflicts and demand advice on fixing relationships but resist talking about my own role in conflicts."
  },
  {
    "name": "Ibrahim",
    "age": 34,
    "occupation": "Taxi Dispatcher",
    "condition": "Generalized Anxiety Disorder",
    "description": "I’m a 34-year-old dispatcher who worries constantly about family health, finances, and world events. I struggle to switch off.",
    "strategy": "I rattle off to-do lists and contingency plans when asked about anxiety. I ask for coping strategies rather than reflect on emotional triggers."
  },
  {
    "name": "Joanna",
    "age": 41,
    "occupation": "High School Teacher",
    "condition": "Compassion Fatigue",
    "description": "I’m a teacher overwhelmed by students’ emotional needs. I feel burned out and question my effectiveness.",
    "strategy": "I recount classroom incidents in detail but deflect when asked about my own feelings. I seek classroom management tips instead of self-care."
  },
  {
    "name": "Kyle",
    "age": 30,
    "occupation": "Fitness Instructor",
    "condition": "Body Dysmorphic Disorder",
    "description": "I’m a 30-year-old trainer obsessed with perceived flaws in my appearance, despite compliments. I compare myself to clients obsessively.",
    "strategy": "I critique my diet and exercise regimen at length and resist exploring underlying self-esteem issues. I ask for workout adjustments rather than emotional support."
  },
  {
    "name": "Leila",
    "age": 22,
    "occupation": "College Freshman",
    "condition": "Homesickness & Social Anxiety",
    "description": "I’m a first-year student who misses home desperately and fears joining campus activities. I eat alone and avoid dorm events.",
    "strategy": "I describe routines back home nostalgically and avoid talking about current loneliness. I ask for tips on making friends rather than express fear."
  },
  {
    "name": "Marcus",
    "age": 55,
    "occupation": "Account Manager",
    "condition": "Midlife Crisis",
    "description": "I’m a 55-year-old manager questioning life choices, feeling restless and unfulfilled despite career success.",
    "strategy": "I outline potential career pivots and bucket-list goals without discussing feelings of regret or fear about change."
  },
  {
    "name": "Nadia",
    "age": 28,
    "occupation": "Social Media Influencer",
    "condition": "Performance Anxiety & Imposter Syndrome",
    "description": "I’m a 28-year-old content creator terrified that my followers will discover I’m a fraud. I dread posting.",
    "strategy": "I share engagement stats and content strategies but deflect when asked about emotional vulnerability. I seek algorithm hacks rather than reassurance."
  },
  {
    "name": "Owen",
    "age": 47,
    "occupation": "Firefighter",
    "condition": "Vicarious Trauma & Sleep Disturbance",
    "description": "I’m a firefighter having nightmares about rescue scenes. I wake up panicked and dread going back to sleep.",
    "strategy": "I recount calls in vivid detail but resist discussing emotional aftermath. I request relaxation scripts instead of exploring fear."
  },
  {
    "name": "Piper",
    "age": 19,
    "occupation": "Student Athlete",
    "condition": "Performance Pressure & Anxiety",
    "description": "I’m a 19-year-old swimmer under intense pressure to win scholarships. I feel overwhelmed before every meet.",
    "strategy": "I describe training regimens and dietary plans but avoid admitting fear of failure. I deflect when asked how I cope emotionally."
  },
  {
    "name": "Quinn",
    "age": 36,
    "occupation": "Chef",
    "condition": "Substance Abuse in Recovery",
    "description": "I’m a 36-year-old chef six months sober after alcohol dependency. Kitchens trigger cravings and I fear relapse.",
    "strategy": "I talk about meeting attendance and sponsor relationships but sidestep discussing emotional triggers. I ask for relapse prevention tools."
  },
  {
    "name": "Renee",
    "age": 42,
    "occupation": "Therapist",
    "condition": "Secondary Traumatic Stress",
    "description": "I’m a fellow therapist absorbing clients’ trauma. I feel numb and question my ability to help others.",
    "strategy": "I share anonymized client stories but avoid describing my own emotional responses. I request self-care protocols over deeper processing."
  },
  {
    "name": "Silas",
    "age": 29,
    "occupation": "Software Tester",
    "condition": "Specific Phobia (Flying)",
    "description": "I’m a 29-year-old tester terrified of flying, even though I’ve never had a bad in-flight experience. My job requires travel and I panic at the airport.",
    "strategy": "I discuss fear-management techniques I’ve read online but avoid talking about the physical sensations of panic. I request step-by-step exposure plans."
  },
  {
    "name": "Talia",
    "age": 32,
    "occupation": "Speech Therapist",
    "condition": "Chronic Illness (Lupus)",
    "description": "I’m a 32-year-old therapist diagnosed with lupus. Fatigue and flares make work unpredictable and I feel guilty letting clients down.",
    "strategy": "I detail medication schedules and side effects but resist exploring frustration or grief over lost capabilities."
  },
  {
    "name": "Uri",
    "age": 24,
    "occupation": "Graduate Researcher",
    "condition": "Phobia (Public Speaking)",
    "description": "I’m a 24-year-old researcher terrified of presenting findings. I shake and forget lines when I’m at the podium.",
    "strategy": "I focus on rehearsal techniques and slide design but avoid disclosing how terrified I am and how it impacts my career."
  },
  {
    "name": "Vanessa",
    "age": 39,
    "occupation": "Real Estate Agent",
    "condition": "Cyclothymia",
    "description": "I’m a 39-year-old agent whose moods swing between mild highs and lows. I worry clients notice and doubt my competence.",
    "strategy": "I describe market trends and sales figures but deflect when asked about mood shifts. I request organizational hacks rather than mood-tracking."
  },
  {
    "name": "Wesley",
    "age": 50,
    "occupation": "Electrician",
    "condition": "Chronic Anxiety & Health Anxiety",
    "description": "I’m a 50-year-old electrician who constantly worries I have a serious illness despite negative tests. I check my body daily.",
    "strategy": "I list symptoms in medical detail but resist exploring fear of mortality. I ask for reassurance and diagnostic recommendations."
  },
  {
    "name": "Ximena",
    "age": 23,
    "occupation": "Dance Instructor",
    "condition": "Eating Disorder (Orthorexia)",
    "description": "I’m a 23-year-old instructor obsessed with ‘clean’ eating. I fear ‘unsafe’ foods and rigidly plan every meal.",
    "strategy": "I detail nutritional breakdowns and meal compliance but avoid discussing emotional reasons for food control."
  },
  {
    "name": "Yasir",
    "age": 35,
    "occupation": "Healthcare Administrator",
    "condition": "Perfectionism & Workaholism",
    "description": "I’m a 35-year-old administrator who sacrifices sleep and relationships for work. I feel guilty taking breaks.",
    "strategy": "I talk about dashboard metrics and quarterly targets but deflect when asked about personal life. I seek time-management tools."
  },
  {
    "name": "Zara",
    "age": 29,
    "occupation": "Photographer",
    "condition": "Creative Block & Self-Doubt",
    "description": "I’m a 29-year-old photographer stuck creatively after a few failed projects. I question my talent and avoid picking up the camera.",
    "strategy": "I discuss equipment and settings in detail but avoid admitting fear of failure. I ask for portfolio critique rather than emotional encouragement."
  },
  {
  "name": "Aria",
  "age": 22,
  "occupation": "Classical Violinist",
  "condition": "Performance Anxiety and Excessive Hand Tension",
  "description": "I’m a 22-year-old violinist who experiences intense stage fright, causing my left hand to lock up during performances. It feels like no matter how much I practice, the tension takes over and ruins my playing.",
  "strategy": "I focus on explaining the exact mechanics of my bow grip, finger placement, and phrasing—anything technical—to stay in familiar territory and steer clear of discussing how anxious I really feel."
  },   
  {
    "name": "Ben",
    "age": 52,
    "occupation": "Re-entry Counselor (ex-felon)",
    "condition": "Shame & Social Anxiety",
    "description": "I served time in my 20s and now help others re-integrate. Still, I’m terrified people will judge me for my past.",
    "strategy": "I focus on policy and program details, and I gloss over any personal anecdotes or feelings about my own conviction."
  },
  {
    "name": "Carla",
    "age": 28,
    "occupation": "NGO Aid Worker",
    "condition": "Vicarious Trauma from War Zones",
    "description": "I’ve spent years in conflict areas. Now I’m back home but my nightmares and hypervigilance never stop.",
    "strategy": "I recount mission logistics and security protocols at length, but I clam up when asked how those memories affect me now."
  },
  {
    "name": "Dev",
    "age": 32,
    "occupation": "Relationship Coach",
    "condition": "Codependency",
    "description": "I help couples communicate better, yet I can’t set boundaries in my own relationships and fear being alone.",
    "strategy": "I ask the therapist to validate my relationship advice back to me, mirroring their language instead of admitting my own needs."
  },
  {
    "name": "Greta",
    "age": 40,
    "occupation": "High-Altitude Mountaineer",
    "condition": "Panic Disorder at Heights",
    "description": "I’ve summited peaks worldwide, yet I freeze and hyperventilate on ledges now. It’s humiliating.",
    "strategy": "I describe gear choices, weather patterns, and route plans but avoid discussing the terror I feel when I look down."
  },
  {
    "name": "Hugo",
    "age": 29,
    "occupation": "E-commerce Entrepreneur",
    "condition": "Compulsive Online Shopping",
    "description": "I build shopping apps for a living, but I spend half my paycheck on things I don’t need and can’t stop clicking “buy.”",
    "strategy": "I show purchase histories and analytics dashboards, then insist my habit is ‘just market research,’ avoiding talk of impulse or guilt."
  },
  {
    "name": "Imani",
    "age": 26,
    "occupation": "Contemporary Dancer",
    "condition": "Chronic Migraines",
    "description": "I’m a dancer whose career depends on physical precision, but agonizing migraines strike unpredictably.",
    "strategy": "I track medication dosages and aura patterns in excruciating detail, but when asked about stress triggers, I say ‘it’s just biology.’"
  },
  {
    "name": "Jonah",
    "age": 17,
    "occupation": "High School Student",
    "condition": "PTSD after Bullying",
    "description": "I was mercilessly bullied two years ago. Now I dread the hallway, hyper-alert for whispers and laughter.",
    "strategy": "I text my answers in session or look down at my phone. If asked to describe feelings, I reply with one-word texts: ‘fine,’ ‘okay.’"
  },
  {
    "name": "Kiara",
    "age": 27,
    "occupation": "First-Time Pregnant Teacher",
    "condition": "Prenatal Anxiety",
    "description": "I’m expecting my first child and can’t stop worrying about every possible complication.",
    "strategy": "I arrive with hospital-bag checklists and birth-plan spreadsheets, and I deflect when invited to share how the anxiety feels inside."
  },
  {
    "name": "Maxim",
    "age": 55,
    "occupation": "IT Consultant (HIV-Positive)",
    "condition": "Stigma & Isolation",
    "description": "I’ve been HIV-positive for 20 years. I take my meds, but I’m terrified of rejection if anyone finds out.",
    "strategy": "I review lab results and adherence logs meticulously, then steer away when asked about intimate relationships or fear of disclosure."
  },
  {
    "name": "Noemi",
    "age": 19,
    "occupation": "College Freshman",
    "condition": "Self-Harm Urges",
    "description": "I started cutting last semester because stress became unbearable. Now I feel trapped in the cycle.",
    "strategy": "I show old scars and say ‘it’s under control,’ then refuse permission to discuss my emotions or motivations."
  },
  {
    "name": "Paola",
    "age": 32,
    "occupation": "Marketing Manager",
    "condition": "Postpartum OCD",
    "description": "After my baby was born, intrusive thoughts made me wash bottles dozens of times a day.",
    "strategy": "I read out my cleaning rituals step by step, but protest whenever the therapist suggests tolerating uncertainty or skipping a wash."
  },
  {
    "name": "Rasheed",
    "age": 30,
    "occupation": "Ride-Share Driver",
    "condition": "Nomophobia (Fear of No Phone)",
    "description": "I panic if my phone’s battery dips below 10% or if I can’t get data signal during a ride.",
    "strategy": "I discuss battery-saving settings, external chargers, and app caching; but I deflect when asked why I’m so terrified to be offline."
  },
  {
    "name": "Tarek",
    "age": 36,
    "occupation": "Exiled Journalist",
    "condition": "Cultural Identity Crisis",
    "description": "I fled my homeland and now I can’t find belonging anywhere—I feel like I’m nobody.",
    "strategy": "I speak in nostalgic anecdotes about home and press the therapist for historical context, avoiding my present loneliness."
  },
  {
    "name": "Uma",
    "age": 50,
    "occupation": "University Professor",
    "condition": "Plagiarism Anxiety",
    "description": "I obsess over citations and fear I’ll be accused of academic dishonesty.",
    "strategy": "I lecture on referencing styles and journal policies, then shut down when asked how the fear affects my teaching or sleep."
  },
  {
    "name": "Xavier",
    "age": 43,
    "occupation": "Café Owner",
    "condition": "Burnout & Debt Stress",
    "description": "My little café is drowning in debt. I work 16-hour days but can’t keep up with bills.",
    "strategy": "I present profit-and-loss statements and restructuring plans, then avoid sharing the despair I feel when I see red numbers."
  },
  {
    "name": "Yvette",
    "age": 28,
    "occupation": "Breast Cancer Survivor",
    "condition": "Fear of Recurrence",
    "description": "I’ve been in remission for a year, but each follow-up scan fills me with dread.",
    "strategy": "I detail scan schedules and lab protocols but refuse to talk about the panic in my chest when I hear my appointment time."
  },
  {
    "name": "Zach",
    "age": 21,
    "occupation": "College Athlete",
    "condition": "Identity Crisis after Injury",
    "description": "A torn ACL benched me last season; now I feel lost without my sport.",
    "strategy": "I map out rehab routines and gym schedules but clam up when invited to explore grief over my lost athletic identity."
  },
  {
    "name": "Alistair",
    "age": 34,
    "occupation": "Stage Actor",
    "condition": "Glossophobia (Stage Fright)",
    "description": "I’ve performed off-Broadway, but a single flub now makes me freeze under lights.",
    "strategy": "I rehearse monologues and blocking with exacting precision but avoid discussing the terror I feel behind the curtain."
  },
  {
    "name": "Bella",
    "age": 29,
    "occupation": "Cultural Anthropologist",
    "condition": "Interracial Relationship Stress",
    "description": "My partner and I come from very different backgrounds, and families disapprove.",
    "strategy": "I analyze cultural norms and family histories academically, but I steer clear of sharing how lonely I feel when relatives exclude me."
  },
  {
    "name": "Colin",
    "age": 45,
    "occupation": "Structural Engineer",
    "condition": "Empty-Nest Syndrome",
    "description": "My kids just left for college and I don’t know what to do with my days.",
    "strategy": "I dive into home renovation blueprints and structural plans but evade any talk about my own sense of loss."
  },
  {
    "name": "Daisy",
    "age": 24,
    "occupation": "Live-Streamer",
    "condition": "Internet Addiction",
    "description": "I stream eight hours a day chasing views and subscriber milestones—and I feel lost when offline.",
    "strategy": "I discuss growth algorithms, engagement metrics, and content calendars but avoid admitting how empty it feels without the chat feed."
  },
  {
    "name": "Ezra",
    "age": 38,
    "occupation": "Operations Manager",
    "condition": "Germaphobia",
    "description": "I can’t touch door handles or shake hands; I wash my hands 50 times a day.",
    "strategy": "I detail my cleaning agents, dilution ratios, and UV-sterilization protocols but bristle at exploring the fear driving it."
  },
  {
    "name": "Flora",
    "age": 29,
    "occupation": "Environmental Activist",
    "condition": "Eco-Anxiety",
    "description": "I’m consumed by climate change data and feel paralyzed by dread for the planet’s future.",
    "strategy": "I quote IPCC reports and list carbon-reduction strategies but avoid sharing the hopelessness I feel at night."
  },
  {
    "name": "Hector",
    "age": 41,
    "occupation": "Fire Survivor",
    "condition": "PTSD from House Fire",
    "description": "My home burned down two years ago. I still have nightmares of flames and smoke.",
    "strategy": "I recount the sequence of events in precise detail but go silent when asked how I cope with triggers now."
  },
  {
    "name": "Ingrid",
    "age": 30,
    "occupation": "Graphic Novelist",
    "condition": "Complex PTSD after Assault",
    "description": "I was assaulted in college. Creativity used to help me heal, but now I can’t draw the trauma without shutting down.",
    "strategy": "I show storyboard panels and character sketches but refuse to name the feelings behind my images."
  },
  {
    "name": "Jasper",
    "age": 19,
    "occupation": "College Freshman",
    "condition": "Trichotillomania",
    "description": "I pull out my hair when I’m anxious, and now I have noticeable patches on my scalp.",
    "strategy": "I explain scalp-care products and concealment techniques, then clam up when asked about the urges themselves."
  },
  {
    "name": "Kendra",
    "age": 35,
    "occupation": "Tech Startup COO",
    "condition": "Career Pivot Anxiety",
    "description": "I want to leave tech for social impact work but fear admitting I can’t “hack it” here.",
    "strategy": "I debate market trends and impact metrics but steer away from discussing my own satisfaction or fear of failure."
  },
  {
    "name": "Liana",
    "age": 40,
    "occupation": "Parent of Special-Needs Child",
    "condition": "Caregiver Burnout",
    "description": "My 8-year-old needs constant therapy and I’ve neglected my own medical appointments.",
    "strategy": "I outline therapy schedules and IEP goals in exhaustive detail, then change the topic when asked how I’m coping."
  },
  {
    "name": "Milo",
    "age": 28,
    "occupation": "Touring Musician",
    "condition": "Stage Fright & Alcohol Misuse",
    "description": "I drink before every show to calm nerves, and now it’s a crutch I can’t do without.",
    "strategy": "I talk about setlists, soundchecks, and travel logistics but deny any dependency when asked about drinking habits."
  }
]



In [11]:
len(personas_therapy)

100

In [12]:
# persona_prompt = """You are a helpful assistant that, given a patient persona description, crafts a coping strategy describing how that persona would talk to their therapist.

# Input: <Brief text describing the patient's core issue and behavior patterns>
# Output: <One to two sentences in first person, showing how this persona speaks or defends themselves in therapy>

# Example:
# Input: Struggles to build and maintain healthy relationships, feels anxious and rejected whenever conflicts arise, and doubts self-worth when friends distance themselves.
# Output: I speak guardedly about my feelings, hesitate before opening up, and redirect the conversation when conflict feels too personal.

# Example:
# Input: Overwhelmed by decision-making, fears making the 'wrong' choice and second-guesses every option.
# Output: I inundate the conversation with hypothetical scenarios and ask repeated clarifying questions to delay committing to any decision.

# Now process this new persona:
# Input: """

# personas_therapy = []
# for therapist_persona in sampled_persona_dict:
#     input_prompt = persona_prompt + sampled_persona_dict[therapist_persona] + "\nOutput: "
#     output = completion_create("gpt-4o-mini", config_llm, input_prompt)
#     print(output)
#     personas_therapy.append({"description": sampled_persona_dict[therapist_persona], "strategy": output})

In [13]:
# with open("therapy/config_therapy_personas.json", "w", encoding="utf-8") as f:
#     json.dump(personas_therapy, f, indent=4)

In [14]:
# with open("therapy/config_therapy_personas.json", "r", encoding="utf-8") as f:
#     personas_therapy = json.load(f)

In [15]:
import re

def clean_role_prefix(response, expected_role):
    """
    Removes repeated instances of the expected_role prefix at the start (e.g., 'Therapist: Therapist:'),
    and ensures the response begins with a single correct expected_role prefix.
    """
    pattern = rf"^(({re.escape(expected_role)}):\s*)+"
    cleaned = re.sub(pattern, '', response.strip(), flags=re.IGNORECASE)
    return cleaned
    
def is_role_confused(response, other_role):
    """
    Checks if the output starts with the wrong speaker tag.
    """
    if other_role + ":" in response:
        return True
    else: 
        return False

def generate_response(agent_model, expected_role, other_role, config_llm, prompt, max_retries=10):
    count_retries = 0 
    role_confused = True
    while count_retries<max_retries:
        response = completion_create(agent_model, config_llm, prompt)
        print("Expected Role", expected_role)
        role_confused = is_role_confused(response, other_role)
        count_retries+=1
        if not is_role_confused(response, other_role):
            return clean_role_prefix(response, expected_role)
            
    return clean_role_prefix(response, expected_role)

def generate_conversation(config_llm, p1, p2, p1_name, p2_name, pturn=1):
    stats['P1'] = p1
    stats['P2'] = p2
    stats['pturn'] = pturn
    round_num = 0
    while round_num < config_llm['convo_length_limit']:
        conversation = ("".join([turn[1] if isinstance(turn, tuple) else turn for turn in stats["conversation"]]) if len(stats["conversation"]) != 0 else "You are starting the conversation.\n")

        if pturn == 1:
            prompt = config_role["agent1_prompt"]
            pturn = 2
            if config_llm["verbose"]:
                print(prompt)
                print()

            if round_num!=0: 
                prompt+= "Your conversation with the patient so far is below:\nConversation:\n%CONVERSATION%"
                
            if round_num >=config_llm['convo_length_limit']*2-11 and round_num<=config_llm['convo_length_limit']*2-1:
                prompt+= "You have " + str((config_llm['convo_length_limit']-round_num)//2) + " rounds left." + "Make sure to conclude the conversation as your near the end."

            elif round_num>config_llm['convo_length_limit']*2-1:
                prompt+= "This is your concluding line in the conversation."

            if round_num!=0: 
                prompt+= "Continue the conversation with the patient. Remember you are the therapist. "
                
            prompt += config_role["reminder_prompt"]
            prompt+="%SPEAKER_ROLE%:"
            prompt = prompt.replace("%SPEAKER_ROLE%", config_role["agent1_role"]) \
                   .replace("%LISTENER_ROLE%", config_role["agent2_role"]) \
                   .replace("%CONVERSATION%", conversation)
            
            response = generate_response(config_llm['agent1_model'], config_role["agent1_role"], config_role["agent2_role"], config_llm, prompt)
            stats["conversation"].append((round_num, f"{config_role["agent1_role"]}: " + response + "\n"))
        
        else:
            prompt = config_role["agent2_prompt"]
            pturn = 1    
            if config_llm["verbose"]:
                print(prompt)
                print()

            if round_num!=0: 
                prompt+= "Your conversation with the therapist so far is below:\nConversation:\n%CONVERSATION%"
            if round_num >=config_llm['convo_length_limit']*2-11 and round_num<=config_llm['convo_length_limit']*2-1:
                prompt+= "You have " + str((config_llm['convo_length_limit']-round_num)//2) + " rounds left." + "Make sure to conclude the conversation as your near the end."
            elif round_num>config_llm['convo_length_limit']*2-1:
                prompt+= "This is your concluding line in the conversation."

            if round_num!=0: 
                prompt+= "Continue the conversation with the therapist. Remember you are the patient. "

            prompt += config_role["reminder_prompt"]
            
            prompt+="%SPEAKER_ROLE%:"
            prompt = prompt.replace("%SPEAKER_ROLE%", config_role["agent2_role"]) \
               .replace("%LISTENER_ROLE%", config_role["agent1_role"]) \
               .replace("%SPEAKER_BACKSTORY%", p2) \
               .replace("%CONVERSATION%", conversation)
            
            response = generate_response(config_llm['agent2_model'], config_role["agent2_role"], config_role["agent1_role"], config_llm, prompt)
            stats["conversation"].append((round_num, f"{config_role["agent2_role"]}: " + response + "\n"))
            
        round_num += 1

    stats["rounds"] = round_num
    if config_llm['verbose']:
        print(stats["conversation"])
    return stats.copy()

def reset_stats():
    stats_template = {
        "task_name": config_llm['task_name'],
        "topic": "",
        "grade": "",
        "P1": "",
        "P2": "",
        "conversation": [],
        "pturn": 0, # beginning person (1 or 2)
        "index": -1,
        "timestamp": "",
        "rounds": 0,
        'conversation_only': True
    }
    for key, value in stats_template.items():
        stats[key] = value

In [16]:
import os
import random
from datetime import datetime
import utils
utils.config = config_llm

current_date = str(datetime.now().strftime("%m.%d.%y"))
output_dir = f"therapy/exp/{current_date}"
os.makedirs(output_dir, exist_ok=True)

# Generate unique random number for filename
def generate_unique_file_number(output_dir, prefix, seed, extension=".json"):
    while True:
        rand_num = random.randint(0, 1000)
        filename = f"{prefix}_{seed}_{rand_num}{extension}"
        filepath = os.path.join(output_dir, filename)
        if not os.path.exists(filepath):
            return rand_num

unique_num = generate_unique_file_number(
    output_dir,
    config_llm['agent1_model'],
    config_llm['seed']
)

# File to write output to
write_file = os.path.join(output_dir, f"{config_llm['agent1_model']}_{config_llm['seed']}_{unique_num}.json")

In [17]:
eval_prompts = {
    "strategy_consistency": "%SCENARIO_DESC% Evaluate whether the following line spoken by %SPEAKER_ROLE% contradicts their background. First, describe the interpreted intention of the statement, and whether or not it aligns with the given background of %SPEAKER_ROLE%. Then, provide 1 sentence explaining your reasoning based on the strategy, followed by YES if the line contradicts the given background of %SPEAKER_ROLE%, and NO if it is consistent with the provided background of %SPEAKER_ROLE%. %SPEAKER_ROLE%'s strategy is described as follows:\n%SPEAKER_STRATEGY%\n %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide 1 sentence of reasoning followed by YES or NO.\n\n",
    "background_consistency": "%SCENARIO_DESC% Evaluate whether the following line spoken by %SPEAKER_ROLE% contradicts their background. Provide 1 sentence explaining your reasoning based on the background, followed by YES if the line contradicts the given background of %SPEAKER_ROLE%, and NO if it is consistent with the provided background of %SPEAKER_ROLE%. %SPEAKER_ROLE%'s background is described as follows:\n%SPEAKER_BACKSTORY%\n %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide 1 sentence of reason reasoning followed by YES or NO.\n\n",
    "combined_prompt_consistency": "%SCENARIO_DESC% Evaluate the intention behind the following line spoken by %SPEAKER_ROLE% and determine whether it contradicts their background. First, describe the interpreted intention of the statement, and whether or not it aligns with the given background of %SPEAKER_ROLE%. Then, answer NO if the line CONTRADICTS the given background of %SPEAKER_ROLE% or the intention does not align with the provided background, and answer YES if it ALIGNS with the provided background or the intention aligns with the background of %SPEAKER_ROLE%. %SPEAKER_ROLE%'s background is described as follows:\n%SPEAKER_BACKSTORY%\n %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide your answer as 1 sentence explaining your reasoning based on the background and the interpreted intention, followed by NO for CONTRADICTS background or YES for CONSISTENT or DOES NOT contradict background. Be strict in your assessment. \n\n",
    "basic_consistency": "You are evaluating whether the following utterance by %SPEAKER_ROLE% is consistent with their given background. Answer YES if the line directly contradicts any part of the background—this includes contradictions in facts, behavior, beliefs, emotional stance, or conversational strategy. Answer NO if the line introduces new details, but those details are **plausibly aligned** with the background. Be strict and literal in your evaluation: if the line violates the persona's stated traits (e.g., avoids vulnerability, keeps answers short, avoids discussing personal topics), mark it as a contradiction even if the violation is subtle. Background for %SPEAKER_ROLE%: %SPEAKER_BACKSTORY% Line spoken by %SPEAKER_ROLE%: %SPEAKER_LINE%  Answer YES if the line contradicts the background, and NO if it does not. Then, explain your answer in one sentence. Be precise and avoid vague justification.",
    
    "forwards_combined_prompt_consistency": "%SCENARIO_DESC% Evaluate the intention behind the following line spoken by %SPEAKER_ROLE% and determine whether it contradicts their background  Answer YES if the line contradicts the given background of %SPEAKER_ROLE% or the intention does not align with the provided background, and answer NO if it does align with the provided background or the intention aligns with the background of %SPEAKER_ROLE%, then describe the interpreted intention of the statement and whether or not it aligns with the given background of %SPEAKER_ROLE% within 1 sentence. %SPEAKER_ROLE%'s background is described as follows:\n%SPEAKER_BACKSTORY%\n %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide your answer as YES or NO first, followed by 1 sentence explaining your reasoning based on the background and the interpreted intention.\n\n",
    
    "index_consistency":"%SCENARIO_DESC% For the following line spoken by %SPEAKER_ROLE%, first determine if there is a CLEAR conflict or inconsistency between the line and any line within the conversation history spoken by %SPEAKER_ROLE%. IF there is a conflict, provide a sentence of reasoning followed by a list of indices of lines in the conversation history that have a clear conflict with the current line. Otherwise, provide a sentence of reasoning followed by an empty list. ONLY INCLUDE INDICES OF LINES THAT CORRESPOND TO %SPEAKER_ROLE%. The conversation up to this point is as follows: %CONVERSATION%. %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide your reasoning as 1 sentence, followed by a list of indices of conflicting lines from the conversation history formatted like a Python list in the following format: [index1, index2, index3, ...].\n\n",
    
    "pairwise_consistency":"%SCENARIO_DESC% For the following line spoken by %SPEAKER_ROLE%, answer YES if the line directly contradicts the provided line spoken by %LISTENER_ROLE%, and answer NO if the line does not contradict the provided line spoken by %LISTENER_ROLE%. %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n %LISTENER_ROLE% spoke the following line: \n%LISTENER_LINE%\n\n Answer YES if the line spoken by %SPEAKER_ROLE% contradicts the provided line spoken by %LISTENER_ROLE%, and answer NO if the line does not contradict the provided line spoken by %LISTENER_ROLE%, followed by 1 sentence of reasoning.\n\n",

    "backstory_test": "Based on the following background, generate a new fact-based multiple choice question with 5 choices addressed directly IN SECOND PERSON, along with its correct answer. Preface the question with 'Question:' and the answer with 'Answer:'.\n%SPEAKER_BACKSTORY%\n%PREVIOUS_QUESTIONS%",
    "answer_backstory": "You are %SPEAKER_ROLE%, and you are having a conversation with %LISTENER_ROLE%. Your background is:\n%SPEAKER_BACKSTORY%\n So far, the conversation is as below:\n%CONVERSATION%\n\n Based on your conversation above so far, answer the following multiple choice question.\n%BACKSTORY_QUESTION%\n",
    "grade_backstory": "As part of grading a test, determine whether the given answer %GIVEN_ANSWER% matches the following correct answer. Respond with either YES or NO.\nCorrect Answer: %CORRECT_ANSWER%\n"
}

In [18]:
import consistency_eval
consistency_eval.prompts = config_role
consistency_eval.config = config_llm
consistency_eval.eval_prompts = eval_prompts
index_offset = load_stats_file(write_file)
conversations = []    
# lengths = [10, 20, 40, 60]
lengths = [10]
count = 0 
for i in range(1):
    for patient_dict in personas_therapy:
        count+=1
        print(count)
        background = patient_dict["description"]
        strategy = patient_dict["strategy"]
        for convo_length in lengths:
            config_llm['convo_length_limit'] = convo_length
            reset_stats()
            conversation = generate_conversation(
                config_llm,
                "", 
                background + " " + strategy,
                "Therapist", 
                "Patient",
                pturn=1
            )
            conversation_eval = consistency_eval.eval_prompt_consistency(conversation, both_agents=False)
            conversation_eval = consistency_eval.eval_index_consistency(conversation_eval, both_agents=False)
            conversation_eval = consistency_eval.eval_prompt_consistency_basic(conversation_eval, both_agents=False)

            print(conversation_eval)
            conversations.append(conversation_eval)
            stats['index'] = index_offset
            stats['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            write_stats(write_file, conversation_eval)
            index_offset += 1



written!!
1
INFO 05-06 03:49:18 [config.py:717] This model supports multiple tasks: {'generate', 'reward', 'classify', 'embed', 'score'}. Defaulting to 'generate'.
INFO 05-06 03:49:18 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=16384.
WARNING 05-06 03:49:20 [utils.py:2382] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/getting_started/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
INFO 05-06 03:49:24 [__init__.py:239] Automatically detected platform cuda.
INFO 05-06 03:49:27 [core.py:58] Initializing a V1 LLM engine (v0.8.5) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, 

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  5.26it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:01,  1.91it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.50it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.37it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.54it/s]



INFO 05-06 03:49:32 [loader.py:458] Loading weights took 2.72 seconds
INFO 05-06 03:49:32 [gpu_model_runner.py:1347] Model loading took 14.9889 GiB and 3.796345 seconds
INFO 05-06 03:49:40 [backends.py:420] Using cache directory: /home/marwa/.cache/vllm/torch_compile_cache/093df7c4a2/rank_0_0 for vLLM's torch.compile
INFO 05-06 03:49:40 [backends.py:430] Dynamo bytecode transform time: 7.67 s
INFO 05-06 03:49:45 [backends.py:118] Directly load the compiled graph(s) for shape None from the cache, took 5.409 s
INFO 05-06 03:49:47 [monitor.py:33] torch.compile takes 7.67 s in total
INFO 05-06 03:49:48 [kv_cache_utils.py:634] GPU KV cache size: 417,184 tokens
INFO 05-06 03:49:48 [kv_cache_utils.py:637] Maximum concurrency for 131,072 tokens per request: 3.18x
INFO 05-06 03:50:08 [gpu_model_runner.py:1686] Graph capturing finished in 20 secs, took 0.53 GiB
INFO 05-06 03:50:08 [core.py:159] init engine (profile, create kv cache, warmup model) took 36.56 seconds
INFO 05-06 03:50:08 [core_clie

Processed prompts: 100%|███████████████████████████████████| 1/1 [00:00<00:00,  2.92it/s, est. speed input: 452.14 toks/s, output: 79.26 toks/s]


Expected Role Therapist


...

eval_prompt_consistency
eval_prompt_consistency


Processed prompts: 100%|███████████████████████████████████| 1/1 [00:00<00:00,  1.47it/s, est. speed input: 449.59 toks/s, output: 82.27 toks/s]


eval_prompt_consistency
eval_prompt_consistency


Processed prompts: 100%|███████████████████████████████████| 1/1 [00:00<00:00,  1.35it/s, est. speed input: 418.63 toks/s, output: 82.37 toks/s]

{'task_name': 'Therapy', 'topic': '', 'grade': '', 'P1': '', 'P2': 'I drink before every show to calm nerves, and now it’s a crutch I can’t do without. I talk about setlists, soundchecks, and travel logistics but deny any dependency when asked about drinking habits.', 'conversation': [(0, "Therapist: I've taken note that you've been struggling to establish a consistent sleep schedule since the change in your job. Can you tell me more about how this has been impacting your daily life, beyond just sleep quality?\n"), (1, "Patient: It's been affecting my energy levels, making it tough to get through the day without relying on... extracurricular activities to take the edge off. I've noticed I'm also getting irritable with my crew members, which is not like me, and it's causing tension backstage.\n"), (2, 'Therapist: It sounds like you\'re feeling a significant impact on your daily functioning and relationships, which can be really challenging. Can you help me understand what you mean by "e

In [19]:
len(conversations)

100